In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from pygeocoder import Geocoder

%matplotlib inline

In [2]:
#calls = pd.read_csv('../Project/Data/911_calls_for_service.csv',index_col='callDateTime',parse_dates=True)
calls = pd.read_csv('../Project/Data/911_calls_for_service.csv')
#calls = calls_full.head(n=10)

In [3]:
calls=calls.drop('Unnamed: 0',axis=1)

In [4]:
type(calls.index)

pandas.core.indexes.range.RangeIndex

In [5]:
calls.priority.isnull().value_counts()

False    2793259
True        6655
Name: priority, dtype: int64

In [6]:
# Can't analyze repeated calls due to masked numbers.
calls.groupby('callNumber')['callNumber'].count().value_counts()

1    2799914
Name: callNumber, dtype: int64

In [7]:
def set_lat(x):
    try:
        return float(x.split(',')[0][1:])
    except:
        return np.nan
    
def set_lon(x):
    try:
        return float(x.split(',')[1][:-1])
    except:
        return np.nan

In [8]:
calls['lat'] = calls['location'].apply(set_lat)

In [9]:
calls['lon'] = calls['location'].apply(set_lon)

In [10]:
calls[calls.lat.isnull()==True].count()

callDateTime        82532
priority            82532
district            82532
description         82532
callNumber          82532
incidentLocation    82532
location            82532
lat                     0
lon                     0
dtype: int64

In [11]:
calls.location = calls.location.replace(to_replace='(,)',value=np.nan)

In [12]:
calls.loc[calls.lat.isnull(),'location'] = calls.loc[calls.lat.isnull(),'incidentLocation']

In [19]:
calls[calls.lat.isnull()==True].head()

,callDateTime,priority,district,description,callNumber,incidentLocation,location,lat,lon
462,2015-07-13 13:07:00,Non-Emergency,SD,Repairs/Service,P151941437,ASCIATE,ASCIATE,NaN,NaN
538,2015-07-13 13:34:00,Non-Emergency,CD,Foot Patrol,P151941534,1500-1700BLK PENNS,1500-1700BLK PENNS,NaN,NaN
539,2015-07-13 13:34:00,Non-Emergency,CD,Foot Patrol,P151941535,1500-1700BLK PENNS,1500-1700BLK PENNS,NaN,NaN
751,2015-07-13 14:45:00,Non-Emergency,NW,Repairs/Service,P151941776,LEWIN SUB,LEWIN SUB,NaN,NaN
1204,2015-07-13 17:20:00,High,SD,Field Interview,P151942362,3100 POTEE ST,3100 POTEE ST,NaN,NaN


In [14]:
calls[(calls.lat.isnull()==True) & (calls.incidentLocation.isnull()==True)]

,callDateTime,priority,district,description,callNumber,incidentLocation,location,lat,lon


In [15]:
def fill_coordinates(x):
    try:
        time.sleep(0.01)
        result = Geocoder.geocode(x)
        if (len(result.coordinates) > 0) & (result.city == 'Baltimore'):
            return result.coordinates
        else:
            return np.nan
    except:
        return "ERROR"

In [200]:
results = Geocoder.geocode('ASCIATE')

In [201]:
results.city

'North Charleston'

In [202]:
type(calls.iloc[0,5])

str

In [22]:
fill_coordinates(calls.iloc[[0,5]])

(39.2896892, -76.6129911)

In [16]:
fill_coordinates(calls[calls.lat.isnull()==True].iloc[1204,5])

(39.3057876, -76.5517868)

In [18]:
fill_coordinates(calls.loc[calls.lon.isnull()==True,'location'].loc[1204])

(39.2494598, -76.6148733)

In [28]:
calls[calls.lat.isnull()==True].iloc[0,5]

'ASCIATE'

In [41]:
fill_coordinates('3100 POTEE ST')

(39.2494598, -76.6148733)

In [ ]:
##### do not run #####
#%timeit
#calls.loc[calls.lon.isnull()==True,'location'] = calls.loc[calls.lon.isnull()==True,'location'].apply(fill_coordinates)
##### do not run #####

In [208]:
calls[calls.lat.isnull()==True].iloc[[0]]

,callDateTime,priority,district,description,callNumber,incidentLocation,location,lat,lon
462,2015-07-13 13:07:00,Non-Emergency,SD,Repairs/Service,P151941437,ASCIATE,NaN,NaN,NaN


In [25]:
calls.loc[calls.lon.isnull()==True]['location'].value_counts()

1000 EDISON HW                1183
5600 LOCH RAVEN BD             775
1100 E COLD SPRING LA          677
400 FONT HILL AV               541
3300 W COLD SPRING LA          432
2000 EDGEWOOD ST               365
2200 W COLD SPRING LA          365
3900 PULASKI HW                342
1300 E NORTHERN PW             317
1600 EDISON HW                 306
5600 O DONNELL ST              270
2800 EDISON HW                 244
FRONT                          186
3600 FEDERAL ST                142
2600 WASHINGTON BD             138
1900 E COLD SPRING LA          112
2100 WINDSOR GARDENS LA        109
**GAS**                        106
3200 TIOGA PW                  101
5100 FEDERAL ST                100
3100 PULASKI HW                 95
5500 SINCLAIR LA                85
1600-BLK W NORTH AV             83
I 100N/W NORTHERN PW            82
2500-1/2 HARFORD RD             82
1000 E MC COMAS ST              78
W NORTHERN PW/FALLS RD          77
3800 PULASKI HW                 75
5000 SINCLAIR LA    

In [ ]:
pd.Dataf

In [65]:
def find_addr(x):
    if x == '(,)':
        return np.nan
    else:
        lat = float(x.split(',')[0][1:])
        lon = float(x.split(',')[1][:-1])
        time.sleep(1)
        return Geocoder.reverse_geocode(lat,lon).formatted_address

def find_zip(x):
    if x == '(,)':
        return np.nan
    else:
        lat = float(x.split(',')[0][1:])
        lon = float(x.split(',')[1][:-1])
        return Geocoder.reverse_geocode(lat,lon).postal_code

In [ ]:
%timeit
calls['address']=calls['location'].transform(find_addr)
calls.to_csv('../Project/Data/911_calls_with_addr.csv')
#OVER_QUERY_LIMIT

In [63]:
calls

,priority,district,description,callNumber,incidentLocation,location,lat,lon
callDateTime,,,,,,,,
2015-07-13 10:41:00,Medium,CD,SEE TEXT,P151941002,0 N CALVERT ST,"(39.2899299,-76.6123462)",39.289930,-76.612346
2015-07-13 10:47:00,Medium,CD,911/NO VOICE,P151941003,600 E FAYETTE ST,"(39.2906737,-76.6071600)",39.290674,-76.607160
2015-07-13 10:42:00,Medium,CD,911/NO VOICE,P151941004,200 E BALTIMORE ST,"(39.2898910,-76.6120720)",39.289891,-76.612072
2015-07-13 10:45:00,Low,CD,PRKG COMPLAINT,P151941005,800 PARK AV,"(39.2985163,-76.6184754)",39.298516,-76.618475
2015-07-13 10:46:00,Medium,SW,AUTO THEFT,P151941006,3500 CLIFTON AV,"(39.3112130,-76.6763150)",39.311213,-76.676315
2015-07-13 10:47:00,Medium,ND,FAMILY DISTURB,P151941007,2700 N CALVERT ST,"(39.3208510,-76.6147390)",39.320851,-76.614739
2015-07-13 10:46:00,High,WD,SILENT ALARM,P151941008,2100 W NORTH AV,"(39.3097096,-76.6513109)",39.309710,-76.651311
2015-07-13 10:49:00,Low,SW,AUTO ACCIDENT,P151941010,3100 WILKENS AV,"(39.2756929,-76.6664179)",39.275693,-76.666418
2015-07-13 10:48:00,Medium,NE,FAMILY DISTURB,P151941011,4800 GILRAY DR,"(39.3483090,-76.5768440)",39.348309,-76.576844


In [49]:
calls['zipCode']=calls['location'].transform(find_zip)
#calls.to_csv('../Project/Data/911_calls_with_addrZip.csv')

In [50]:
calls.head()

,priority,district,description,callNumber,incidentLocation,location,address,zipCode
callDateTime,,,,,,,,
2015-07-13 10:41:00,Medium,CD,SEE TEXT,P151941002,0 N CALVERT ST,"(39.2899299,-76.6123462)","One Calvert Plaza, Baltimore, MD 21202, USA",21202
2015-07-13 10:47:00,Medium,CD,911/NO VOICE,P151941003,600 E FAYETTE ST,"(39.2906737,-76.6071600)","560-618 E Fayette St, Baltimore, MD 21202, USA",21202
2015-07-13 10:42:00,Medium,CD,911/NO VOICE,P151941004,200 E BALTIMORE ST,"(39.2898910,-76.6120720)","200 E Baltimore St, Baltimore, MD 21202, USA",21202
2015-07-13 10:45:00,Low,CD,PRKG COMPLAINT,P151941005,800 PARK AV,"(39.2985163,-76.6184754)","800-808 Park Ave, Baltimore, MD 21201, USA",21201
2015-07-13 10:46:00,Medium,SW,AUTO THEFT,P151941006,3500 CLIFTON AV,"(39.3112130,-76.6763150)","3500 Clifton Ave, Baltimore, MD 21216, USA",21216
